In [126]:
# Make division default to floating-point, saving confusion
from __future__ import division
from __future__ import print_function

# Allowed libraries 
import numpy as np
import pandas as pd
import scipy as sp
import scipy.special
import heapq as pq
import matplotlib as mp
import matplotlib.pyplot as plt
import math
from itertools import product, combinations
from collections import OrderedDict as odict
import collections
from graphviz import Digraph, Graph
from tabulate import tabulate
import copy
import sys
import os
import datetime
import sklearn
import ast
import re

In [81]:
# read data
data = pd.read_csv('data.csv')
r16 = data[['time']]

# shift values down to get X_t-1
r16['X_t-1'] = data['r16'].shift(1)
r16['X_t'] = data['r16']
r16['s1'] = data['reliable_sensor1']

# drop first row (X_1)
r16.drop(r16.index[0], inplace=True)
r16['X_t-1'] = r16['X_t-1'].astype('int64')

# chance values in X_t to 1 if > 1
# motion to 1, no motion to 0
r16.loc[r16['X_t'] > 1, 'X_t'] = 1
r16.loc[r16['X_t-1'] > 1, 'X_t-1'] = 1
r16.loc[r16['s1'] == 'no motion', 's1'] = 0
r16.loc[r16['s1'] == 'motion', 's1'] = 1

r16.head()

,time,X_t-1,X_t,s1
1,08:00:30,0,0,0
2,08:00:45,0,0,0
3,08:01:00,0,0,0
4,08:01:15,0,0,0
5,08:01:30,0,0,0


In [130]:
# calculate transition probabilities P(X_t | X_t-1)
on = r16[r16['X_t-1'] == 1].drop(['time','s1'], axis=1)
print("P(off | on) =", np.round(on['X_t'].value_counts()[0] / len(on), 4))
print("P(on | on) =", np.round(on['X_t'].value_counts()[1] / len(on), 4))

off = r16[r16['X_t-1'] == 0].drop(['time','s1'], axis=1)
print("P(off | off) =", np.round(off['X_t'].value_counts()[0] / len(off), 4))
print("P(on | off) =", np.round(off['X_t'].value_counts()[1] / len(off), 4))


P(off | on) = 0.0126
P(on | on) = 0.9874
P(off | off) = 0.9876
P(on | off) = 0.0124


In [129]:
# calculate emission probabilities P(s1 | X_t)
on = r16[r16['X_t'] == 1].drop(['time','X_t-1'], axis=1)
print("P(no motion | on) =", np.round(on['s1'].value_counts()[0] / len(on), 4))
print("P(motion | on) =", np.round(on['s1'].value_counts()[1] / len(on), 4))

off = r16[r16['X_t'] == 0].drop(['time','X_t-1'], axis=1)
print("P(no motion | off) =", np.round(off['s1'].value_counts()[0] / len(off), 4))
print("P(motion | off) =", np.round(off['s1'].value_counts()[1] / len(off), 4))

P(no motion | on) = 0.016
P(motion | on) = 0.984
P(no motion | off) = 0.9735
P(motion | off) = 0.0265


In [132]:
# CODE TAKEN FROM WEEK 5 TUTORIAL

def printFactor(f):
    """
    argument 
    `f`, a factor to print on screen
    """
    # Create a empty list that we will fill in with the probability table entries
    table = list()
    
    # Iterate over all keys and probability values in the table
    for key, item in f['table'].items():
        # Convert the tuple to a list to be able to manipulate it
        k = list(key)
        # Append the probability value to the list with key values
        k.append(item)
        # Append an entire row to the table
        table.append(k)
    # dom is used as table header. We need it converted to list
    dom = list(f['dom'])
    # Append a 'Pr' to indicate the probabity column
    dom.append('Pr')
    print(tabulate(table,headers=dom,tablefmt='orgtbl'))

def prob(factor, *entry):
    """
    argument 
    `factor`, a dictionary of domain and probability values,
    `entry`, a list of values, one for each variable in the same order as specified in the factor domain.
    
    Returns p(entry)
    """

    return factor['table'][entry]       

def join(f1, f2, outcomeSpace):
    """
    argument 
    `f1`, first factor to be joined.
    `f2`, second factor to be joined.
    `outcomeSpace`, dictionary with the domain of each variable
    
    Returns a new factor with a join of f1 and f2
    """
    
    # First, we need to determine the domain of the new factor. It will be union of the domain in f1 and f2
    # But it is important to eliminate the repetitions
    common_vars = list(f1['dom']) + list(set(f2['dom']) - set(f1['dom']))
    
    # We will build a table from scratch, starting with an empty list. Later on, we will transform the list into a odict
    table = list()
    
    # Here is where the magic happens. The product iterator will generate all combinations of varible values 
    # as specified in outcomeSpace. Therefore, it will naturally respect observed values
    for entries in product(*[outcomeSpace[node] for node in common_vars]):
        
        # We need to map the entries to the domain of the factors f1 and f2
        entryDict = dict(zip(common_vars, entries))
        f1_entry = (entryDict[var] for var in f1['dom'])
        f2_entry = (entryDict[var] for var in f2['dom'])
        
        # Use the fuction prob to calculate the probability 
        p1 = prob(f1, *f1_entry)           
        p2 = prob(f2, *f2_entry)           
        
        # Create a new table entry with the multiplication of p1 and p2
        table.append((entries, p1 * p2))
    return {'dom': tuple(common_vars), 'table': odict(table)}


def marginalize(f, var, outcomeSpace):
    """
    argument 
    `f`, factor to be marginalized.
    `var`, variable to be summed out.
    `outcomeSpace`, dictionary with the domain of each variable
    
    Returns a new factor f' with dom(f') = dom(f) - {var}
    """    
    
    # Let's make a copy of f domain and convert it to a list. We need a list to be able to modify its elements
    new_dom = list(f['dom'])
    
    new_dom.remove(var)       # Remove var from the list new_dom 
    table = list()            # Create an empty list for table
    for entries in product(*[outcomeSpace[node] for node in new_dom]):
        s = 0;                     # Initialize the summation variable s. 
        # We need to iterate over all possible outcomes of the variable var
        for val in outcomeSpace[var]:
            # To modify the tuple entries, we will need to convert it to a list
            entriesList = list(entries)
            # We need to insert the value of var in the right position in entriesList
            entriesList.insert(f['dom'].index(var), val)
            # Calculate the probability of factor f for entriesList. 
            p = prob(f, *tuple(entriesList))   
            # Sum over all values of var by accumulating the sum in s.  
            s = s + p                            
            
        # Create a new table entry with the multiplication of p1 and p2
        table.append((entries, s))
    return {'dom': tuple(new_dom), 'table': odict(table)}

def evidence(var, e, outcomeSpace):
    """
    argument 
    `var`, a valid variable identifier.
    `e`, the observed value for var.
    `outcomeSpace`, dictionary with the domain of each variable
    
    Returns dictionary with a copy of outcomeSpace with var = e
    """    
    # Make a copy of outcomeSpace
    newOutcomeSpace = outcomeSpace.copy()      
    # Replace the domain of variable var with a tuple with a single element e
    newOutcomeSpace[var] = (e,)                
    return newOutcomeSpace

def normalize(f):
    """
    argument 
    `f`, factor to be normalized.
    
    Returns a new factor f' as a copy of f with entries that sum up to 1
    """ 
    table = list()
    sum = 0
    for k, p in f['table'].items():
        sum = sum + p
    for k, p in f['table'].items():
        table.append((k, p/sum))
    return {'dom': f['dom'], 'table': odict(table)}

def maximize(f, var, outcomeSpace):
    """
    argument 
    `f`, factor to be marginalized.
    `var`, variable to be maximized out.
    `outcomeSpace`, dictionary with the domain of each variable
    
    Returns a new factor f' with dom(f') = dom(f) - {var}
    """    
    # Let's make a copy of f domain and convert it to a list. We need a list to be able to modify its elements
    new_dom = list(f['dom'])
    new_dom.remove(var)            # Remove var from the list new_dom
    table = list()                 # Create an empty list for table.
    for entries in product(*[outcomeSpace[node] for node in new_dom]):     
        m = 0;                  # Initialize the maximization variable m.

        # We need to iterate over all possible outcomes of the variable var
        for val in outcomeSpace[var]:
            # To modify the tuple entries, we will need to convert it to a list
            entriesList = list(entries)
            # We need to insert the value of var in the right position in entriesList
            entriesList.insert(f['dom'].index(var), val)
            # Calculate the probability of factor f for entriesList.
            p = prob(f, *tuple(entriesList))     
            # Maximize over all values of var by storing the max value in m.
            m = max(m, p)                        
            
        # Create a new table entry with the multiplication of p1 and p2
        table.append((entries, m))
    return {'dom': tuple(new_dom), 'table': odict(table)}

In [140]:
# CODE TAKEN FROM WEEK 5 TUTORIAL

def viterbiOnline(f, transition, emission, stateVar, emissionVar, emissionEvi, outcomeSpace, norm):
    """
    argument 
    `f`, factor that represents the previous state.
    `transition`, transition probabilities from time t-1 to t.
    `emission`, emission probabilities.
    `stateVar`, state (hidden) variable.
    `emissionVar`, emission variable.
    `emissionEvi`, emission observed evidence. If undef, we do only the time update
    `outcomeSpace`, dictionary with the domain of each variable.
    
    Returns a new factor that represents the current state.
    """ 

    # Set fCurrent as a copy of f
    fCurrent = f.copy()
    # Set the f_previous domain to be a list with a single variable name appended with '_t-1' to indicate previous time step
    fCurrent['dom'] = (stateVar + '_t-1', )       
    # Make the join operation between fCurrent and the transition probability table    
    fCurrent = join(fCurrent, transition, outcomeSpace)        
    # Eliminate the randVariable_t-1 with the maximization operation
    fCurrent = maximize(fCurrent, fCurrent['dom'][0], outcomeSpace)        
    # If emissionEvi == None, we will assume this time step has no observed evidence    
    if emissionEvi != None:                  # WARNING: do not change this line
        # Set evidence in the form emissionVar = emissionEvi    
        newOutcomeSpace = evidence(emissionVar, emissionEvi, outcomeSpace)     
        # Make the join operation between fCurrent and the emission probability table. Use the newOutcomeSpace    
        fCurrent = join(fCurrent, emission, newOutcomeSpace)      
        # Marginalize emissionVar. Use the newOutcomeSpace    
        fCurrent = marginalize(fCurrent, emissionVar, newOutcomeSpace)         
        # Normalize fcurrent.
        if norm:
            fCurrent = normalize(fCurrent)           
    # Set the domain of w to be name of the random variable without time index
    fCurrent['dom'] = (stateVar, )
    return fCurrent

def viterbiBatch(f, transition, emission, stateVar, emissionVar, emissionEviList, outcomeSpace, norm=True):
    """
    argument 
    `f`, factor that represents the previous state.
    `transition`, transition probabilities from time t-1 to t.
    `emission`, emission probabilities.
    `stateVar`, state (hidden) variable.
    `emissionVar`, emission variable.
    `emissionEviList`, emission observed evidence.
    `outcomeSpace`, dictionary with the domain of each variable.
    
    Returns a new factor that represents the current state.
    """      
    timeLine = []
    # Set fCurrent as a copy of f
    fCurrent = f.copy()
    for emissionEvi in emissionEviList:
        # Call the online version of the Viterbi algorithm to update one time step
        fCurrent = viterbiOnline(fCurrent, transition, emission, stateVar, emissionVar, emissionEvi, outcomeSpace, norm)                 # 1 line
        # Print the current factor for debugging
        timeLine.append(fCurrent)
    return timeLine

def traceBack(timeLine, start, transition, emission, stateVar, emissionVar, emissionEviList, outcomeSpace):
    """
    argument 
    `f`, factor that represents the previous state.
    `transition`, transition probabilities from time t-1 to t.
    `emission`, emission probabilities.
    `stateVar`, state (hidden) variable.
    `emissionVar`, emission variable.
    `emissionEviList`, emission observed evidence.
    `outcomeSpace`, dictionary with the domain of each variable.
    
    Returns an array with the MLE assignment.
    """ 

    t = len(timeLine) - 1
    mleList = [outcomeSpace[stateVar][0]] * (t+1)
    mleList[t] = max(timeLine[t]['table'], key=timeLine[t]['table'].get)[0]
    p = prob(timeLine[t], mleList[t])
    for t in range(t,0,-1):
        p_e = prob(emission, mleList[t], emissionEviList[t])
        for state in outcomeSpace[stateVar]:
            p_t = prob(transition, state, mleList[t])
            if p_t != 0 and p_e!=0:
                if abs(prob(timeLine[t-1], state) - p/p_e/p_t) < 0.00000001:
                    mleList[t-1] = state
                    p = prob(timeLine[t-1], state)
    p_e = prob(emission, mleList[0], emissionEviList[0])
    for state in outcomeSpace[stateVar]:
        p_t = prob(transition, state, mleList[0])
        if p_t != 0 and p_e != 0:       
            if abs(prob(start, state) - p/p_e/p_t) < 0.00000001:
                mleList[0] = state
                p = prob(timeLine[0], state)
    return mleList

In [152]:
outcomeSpaceLights = {
    "Lights_t": ('on','off'),
    "Lights_t-1": ('on','off'),
}

transitionLights = {
    'dom': ('Lights_t-1', 'Lights_t'), 
    'table': odict([
        (('on','on'), 0.9874),
        (('on','off'), 0.0126),
        (('off','on'), 0.0124),
        (('off','off'), 0.9876),        
    ])
}

startLights = {
    'dom': ('Lights',), 
    'table': odict([
        (('on',), 0.0),
        (('off',), 1.0),
    ])
}

evidenceLights = {
    'dom': ('Lights_t', 'Sensor_t'), 
    'table': odict([
        (('on','motion'), 0.984),
        (('on','no motion'), 0.016),
        (('off','motion'), 0.0265),
        (('off','no motion'), 0.9735),
    ])
}

evidenceOutcomes = ('no motion', 'no motion', 'no motion', 'motion', 'motion', 'motion', 'no motion', 'no motion','motion', 'motion')
timeline = viterbiBatch(startLights, transitionLights, evidenceLights, 'Lights', 'Sensor_t', evidenceOutcomes, outcomeSpaceLights, norm=True)
for t in range(len(timeline)):
    print("Time: ", t)
    printFactor(timeline[t])
    print()

mleList = traceBack(timeline, startLights, transitionLights, evidenceLights, 'Lights_t', 'Sensor_t', evidenceOutcomes, outcomeSpaceLights)
for t in range(len(mleList)):
    print("Time: ", t)
    print(mleList[t])
    print()

Time:  0
| Lights   |          Pr |
|----------+-------------|
| on       | 0.000206317 |
| off      | 0.999794    |

Time:  1
| Lights   |          Pr |
|----------+-------------|
| on       | 0.000206317 |
| off      | 0.999794    |

Time:  2
| Lights   |          Pr |
|----------+-------------|
| on       | 0.000206317 |
| off      | 0.999794    |

Time:  3
| Lights   |       Pr |
|----------+----------|
| on       | 0.317974 |
| off      | 0.682026 |

Time:  4
| Lights   |        Pr |
|----------+-----------|
| on       | 0.94538   |
| off      | 0.0546204 |

Time:  5
| Lights   |         Pr |
|----------+------------|
| on       | 0.998446   |
| off      | 0.00155386 |

Time:  6
| Lights   |       Pr |
|----------+----------|
| on       | 0.562932 |
| off      | 0.437068 |

Time:  7
| Lights   |        Pr |
|----------+-----------|
| on       | 0.0207256 |
| off      | 0.979274  |

Time:  8
| Lights   |   Pr |
|----------+------|
| on       | 0.44 |
| off      | 0.56 |

Time:  9
|